<a href="https://colab.research.google.com/github/akamrume328/tennisvision/blob/feature%2F%231/notebooks/2_model_training_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google ColabでのYOLOv8モデルのトレーニング (Google Drive対応版)

このノートブックは、Google Driveをマウントし、Drive上のデータセットを使用してYOLOv8モデルをトレーニングします。  
以下の内容をカバーします：  
1. Google Driveのマウント  
2. `ultralytics`ライブラリのインストール  
3. データセットの準備とパス設定  
4. YOLOv8モデルのトレーニング  

In [ ]:
# Driveマウント → ultralyticsインストール → データ展開 → パス・ハイパーパラ設定
from google.colab import drive; drive.mount('/content/drive', force_remount=True)
!pip install ultralytics

import os, shutil
# データセットをローカルにコピー
drive_data = '/content/drive/MyDrive/datasets/final_merged_dataset'
local_data = '/content/datasets'
if os.path.exists(local_data): shutil.rmtree(local_data)
shutil.copytree(drive_data, local_data)
dataset_yaml_path = os.path.join(local_data, 'data.yaml')

# 出力先
project_output_dir  = '/content/drive/MyDrive/models/YOLOv8_Training_Outputs'
os.makedirs(project_output_dir, exist_ok=True)

# 実験設定
experiment_name      = 'tennis_detection_drive_run'
training_device      = 'cuda:0'
checkpoint_to_resume = None
save_every_n_epochs  = 10

print(f"YAML: {dataset_yaml_path}")
print(f"出力ディレクトリ: {project_output_dir}")
print(f"実験名: {experiment_name}, デバイス: {training_device}")
print(f"チェックポイント毎: {save_every_n_epochs} epoch")

## YOLOv8モデルの初期化とトレーニング

### モデルのロードとトレーニング実行

このセルでは、YOLOv8モデルを初期化し、トレーニングを実行します。

**チェックポイントからの再開:**
前の「パスの設定」セルで `checkpoint_to_resume` に有効なチェックポイントファイル（例: `last.pt` や特定の `epoch_XXX.pt`）のパスが指定されている場合、モデルはそのチェックポイントからロードされ、トレーニングが再開されます。これにより、中断したトレーニングを続けたり、既存のモデルをさらにファインチューニングしたりできます。
`checkpoint_to_resume` が `None` であるか、指定されたパスにファイルが存在しない場合は、`model_name` で指定された新しい事前学習済みモデル（例: `yolov8s.pt`）からトレーニングが開始されます。

**定期的なチェックポイント保存:**
`model.train()` メソッドの `save_period` パラメータに、前のセルで設定した `save_every_n_epochs` の値が渡されます。
`save_every_n_epochs` が1以上の場合、指定されたエポック数ごとにモデルの重みが `project_output_dir/experiment_name/weights/epoch_XXX.pt` という名前で保存されます。これにより、トレーニングの途中経過を細かく保存し、必要に応じて特定のエポックから再開できるようになります。

**その他のパラメータ:**
-   `data`: データセット設定ファイル (`data.yaml`) へのパス。
-   `epochs`: 総トレーニングエポック数。
-   `imgsz`: 入力画像のサイズ。
-   `batch`: バッチサイズ。
-   `project`: トレーニング結果が保存される親ディレクトリ。
-   `name`: 今回のトレーニング実行固有のサブディレクトリ名。
-   `exist_ok=True`: `project/name` ディレクトリが既に存在する場合でもエラーとせず、上書きまたは追記を許可します。
-   `device`: トレーニングに使用するデバイス（例: `'cpu'`, `'cuda:0'`）。前のセルで設定した `training_device` 変数の値が使用されます。

トレーニングが開始されると、進行状況、損失、評価メトリクスなどが表示されます。
**トレーニングが正常に完了した場合、このセルの処理がすべて終了すると、Colabのランタイムは自動的に切断され、リソースが解放されます。エラーが発生した場合は、ランタイムは切断されません。**

In [ ]:
from ultralytics import YOLO
import os
from google.colab import runtime
import torch

# GPU メモリクリア関数を定義
def clear_gpu_memory():
    try:
        import torch
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            print("GPUメモリキャッシュをクリアしました。")
        else:
            print("CUDAが利用できないため、GPUメモリクリアはスキップされました。")
    except Exception as e:
        print(f"GPUメモリクリア中にエラーが発生しました: {e}")

# 事前学習済みのYOLOv8モデルをロードするか、チェックポイントから再開
# 'yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt', 'yolov8x.pt' などのサイズを選択可能
# 初めての場合は 'yolov8n.pt' (Nano) または 'yolov8s.pt' (Small) を推奨
model_name = 'yolov8x.pt' # 新規トレーニングの場合のデフォルトモデル

model_load_path = model_name # デフォルトは新規モデル
if checkpoint_to_resume and os.path.exists(checkpoint_to_resume):
    print(f"チェックポイントからトレーニングを再開します: {checkpoint_to_resume}")
    model_load_path = checkpoint_to_resume
elif checkpoint_to_resume:
    print(f"指定されたチェックポイント '{checkpoint_to_resume}' が見つからなかったため、新しい事前学習済みモデル ({model_name}) からトレーニングを開始します。")
else:
    print(f"新しい事前学習済みモデル ({model_name}) からトレーニングを開始します。")

model = YOLO(model_load_path)

# トレーニングのハイパーパラメータを定義
num_epochs = 150  # トレーニングエポック数 (例: 50, 100, 200)
batch_size = 8   # バッチサイズ (GPU/CPUメモリに応じて調整: 例 8, 16, 32)
img_size = 1920    # 入力画像サイズ (例: 640, 1280)

# オプティマイザ / 学習率
optimizer       = 'AdamW'
lr0             = 1e-3      # 初期学習率
lrf             = 0.05      # cosine LR の最終値（lr_end = lr0 * lrf）
warmup_epochs   = 3.0       # ウォームアップ期間
weight_decay    = 0.01      # L2 正則化

# データ拡張（小物体検出で有効なものを少し強めに）
mosaic          = 0.5       # 50% の確率で Mosaic
mixup           = 0.5       # 50% の確率で MixUp
augment         = True      # その他の augment を有効化

# DataLoader
workers         = 8         # CPU コア数に合わせて


print(f"モデルのトレーニングを開始します。")
print(f"使用デバイス: {training_device}")
print(f"エポック数: {num_epochs}, バッチサイズ: {batch_size}, 画像サイズ: {img_size}")
if save_every_n_epochs > 0:
    print(f"チェックポイントは {save_every_n_epochs} エポックごとに保存されます。")

print("\n重要: トレーニングが正常に完了した場合、このセルの最後にランタイムが自動的に切断されます。")
print("エラーが発生した場合は、ランタイムは切断されず、エラーメッセージを確認できます。")
print(f"トレーニング結果はマウントされたGCSディレクトリ ({project_output_dir}) に保存される設定になっています。")

# トレーニングを開始
training_successful = False
try:
    print("=== トレーニング開始前のGPUメモリクリア ===")
    clear_gpu_memory()

    results = model.train(
        data            = dataset_yaml_path,
        epochs          = num_epochs,
        imgsz           = img_size,
        batch           = batch_size,
        project         = project_output_dir,
        name            = experiment_name,
        exist_ok        = True,
        save_period     = save_every_n_epochs,
        device          = training_device,
        optimizer       = optimizer,
        lr0             = lr0,
        lrf             = lrf,
        warmup_epochs   = warmup_epochs,
        weight_decay    = weight_decay,
        mosaic          = mosaic,
        mixup           = mixup,
        augment         = augment,
        workers         = workers,
        cache           = True,
        amp             = True,  # AMP (自動混合精度) を有効化
    )
    print("\nトレーニングが正常に完了しました！")
    full_output_path = os.path.join(project_output_dir, experiment_name)
    print(f"結果、ログ、モデルの重みはマウントされたGCS上の次の場所に保存されました: {full_output_path}")
    training_successful = True

    print("\nトレーニングが正常に完了したため、ランタイムを切断します...")
    print(f"必要なファイルがGCS ({full_output_path}) に保存されていることを確認してください。")
    print("ランタイム切断中...")
    runtime.unassign()
    print("ランタイムが切断されました。Colabの接続は解除されます。")

except torch.cuda.OutOfMemoryError as e:
    print("GPU OOM エラーが発生しました:", e)
    print("GPUメモリをクリアします...")
    clear_gpu_memory()
    print("バッチサイズを小さくするか、画像サイズを下げることを検討してください。")
except Exception as e:
    print(f"トレーニング中に予期せぬエラーが発生しました: {e}")
    import traceback
    traceback.print_exc()
finally:
    if not training_successful:
        print("\nこのセルの処理は終了しましたが、トレーニング中にエラーが発生したか、正常に完了しませんでした。")
        print(f"エラーが発生した場合でも、部分的な結果がGCS ({os.path.join(project_output_dir, experiment_name)}) に保存されている可能性があります。")

## トレーニング結果と保存されたモデル

トレーニング後、結果（メトリクス、混同行列、モデルの重みなど）は指定したマウントされたGCSディレクトリに保存されます：
`{project_output_dir}/{experiment_name}` (例: `/content/gcs/models/YOLOv8_Training_Outputs/tennis_detection_gcs_run1`)

確認すべき主なファイル：
- **重み:** `weights`サブディレクトリ内 (例: `best.pt`, `last.pt`)
  - `best.pt`: 最良の検証メトリクス（通常はmAP50-95）を達成したモデルの重み。このモデルを推論に使用するのが一般的です。
  - `last.pt`: トレーニングの最終エポックのモデルの重み。
  - `epoch_XXX.pt` (例: `epoch_10.pt`, `epoch_20.pt`): `save_period` で指定されたエポックごとに保存されるチェックポイント。これらを使用して特定のエポックからトレーニングを再開したり、その時点でのモデル性能を評価したりできます。
- **結果CSV:** `results.csv`にはエポックごとのメトリクスの概要が含まれています。
- **プロット:** 混同行列、P-R曲線などのさまざまなプロット (例: `confusion_matrix.png`, `PR_curve.png`)

これらのファイルは自動的にGCSに同期されるため、マウントが解除された後でもGCS上で利用できます。

In [ ]:
import os

# 実験の重みディレクトリ (マウントされたローカルパス)
weights_dir = os.path.join(project_output_dir, experiment_name, 'weights')

print(f"\n重みディレクトリ内のファイルをリストアップします ({weights_dir}):")

if os.path.exists(weights_dir):
    try:
        files_in_weights_dir = os.listdir(weights_dir)
        if files_in_weights_dir:
            for f_name in files_in_weights_dir:
                print(f"- {f_name}")
        else:
            print(f"重みディレクトリ ({weights_dir}) は空です。")
            print("トレーニングが完了していないか、エラーが発生した可能性があります。")
    except Exception as e:
        print(f"重みディレクトリの内容を読み取れませんでした: {e}")
else:
    print(f"\n重みディレクトリが見つかりません: {weights_dir}")
    print("トレーニングが完了していないか、エラーが発生した可能性があります。")

# トレーニング済みモデルの最良の重みへのローカルパス
best_model_path = os.path.join(weights_dir, 'best.pt')
print(f"\n最良のモデルへのローカルパス: {best_model_path}")

if os.path.exists(best_model_path):
    print("最良のモデルファイル (best.pt) が見つかりました。")
    # ファイルサイズも表示
    try:
        file_size = os.path.getsize(best_model_path)
        print(f"ファイルサイズ: {file_size / (1024*1024):.1f} MB")
    except Exception as e:
        print(f"ファイルサイズを取得できませんでした: {e}")
else:
    print("最良のモデルファイル (best.pt) が見つかりません。トレーニングが失敗したか、生成されなかった可能性があります。")

print(f"\n注意: ファイルは自動的にGCSバケット (gs://{bucket_name}) に同期されています。")